In [12]:
import os

In [13]:
class author:
    def __init__(self,id,name) -> None:
        self.id=id
        self.name=name
        self.paper_list=[]
    def append_article(self,name):
        self.paper_list.append(name)

class article:
    def __init__(self,name,id) -> None:
        self.id=id
        self.name=name
        self.author=[]
        self.field=[]
        self.tc=0
        self.wos=''
    def append_wos(self,wos):
        self.wos=wos
    def append_author(self,author):
        self.author.append(author)
    def append_tc(self,tc):
        self.tc=tc
    def append_field(self,field):
        self.field.append(field)
class field:
    def __init__(self,name,id) -> None:
        self.name=name
        self.id=id
fields=[]
fields_id={}
authors=[]
articles=[]
author_id={}
articles_id={}
title_id={}
node_author=[]
edge_title=[]
folder_path=r"data"
files = os.listdir(folder_path)
    # 按照文件名的顺序排序
    # 逐个读取文件

for file in files[0:3]:
    with open(os.path.join(folder_path, file), 'r',encoding='utf-8') as f:
        line=f.readline()
        tmp_author=[]
        tmp_title=[]
        while line:
            if line[0:2]=='AF':
                tmp_author.append(line[3:len(line)-1])
                line=f.readline()
                while line[0:2]=='  ':
                    tmp_author.append(line[3:len(line)-1])
                    line=f.readline()
                for a in tmp_author:
                    if a not in author_id.keys():
                        authors.append(author(len(author_id),a))
                        author_id[a]=len(author_id)
                tmp_title.append(line[3:len(line)-1])
                line=f.readline()
                while line[0:2]=='  ':
                    tmp_title.append(line[3:len(line)-1])
                    line=f.readline()
                title=' '.join(tmp_title)
                title=title.replace('"', "'")
                articles_id[title]=len(articles_id)
                articles.append(article(title,len(articles)))
                for i in tmp_author:
                    articles[len(articles)-1].append_author(i)
                    authors[author_id[i]].append_article(title)
                line=f.readline()
                while line[0:2]!='ER':
                    if line[0:2]=='ID':
                        tmp_fields=line[2:len(line)-1].split("; ")
                        for i in tmp_fields:
                            if i not in fields_id.keys():
                                fields.append(field(i,len(fields)))
                                fields_id[i]=len(fields_id)
                            articles[len(articles)-1].append_field(i)
                    if line[0:2]=='TC':
                        articles[len(articles)-1].append_tc(int(line[3:len(line)-1]))
                    if line[0:2]=='UT':
                        articles[len(articles)-1].append_wos(int(line[7:len(line)-1]))
                    line=f.readline()
                """
                for i in range(0,len(tmp_author)):
                    for j in range(i+1,len(tmp_author)):
                        edge_title.append((author_id[tmp_author[i]],author_id[tmp_author[j]],title))
                """
                tmp_author.clear()
                tmp_title.clear()
            line=f.readline()
   
print(author_id["Luo, Jingwen"])

In [14]:
from neo4j import GraphDatabase

# 创建驱动程序实例
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "LBZlbz12"))

# 创建一个会话

with driver.session() as session:
    # 在事务中运行Cypher查询

  

    session.run("MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r")
    for author in authors:
        session.run(f'CREATE (author:author {{name:"{author.name}",id:{author.id}}})')
    
    for article in articles:
        session.run(f'CREATE (article:article{{name:"{article.name}",id:{article.id},tc:{article.tc},wos:"{article.wos}"}})')

    for field in fields:
        session.run(f'CREATE (field:field {{name:"{field.name}",id:{field.id}}})')
    
    #session.run("MATCH ()-[r]-() DELETE r")

    session.run("CREATE INDEX FOR (n:author) ON (n.name)")
    session.run("CREATE INDEX FOR (n:article) ON (n.name)")
    session.run("CREATE INDEX FOR (n:field) ON (n.name)")
 

    for author in authors:
        for a in author.paper_list:
            session.run(f'MATCH (a:author),(b:article) WHERE "{a}"= b.name AND a.name="{author.name}" CREATE (a)-[r:WRITE]->(b)')
            session.run(f'MATCH (a:author),(b:article) WHERE "{a}"= b.name AND a.name="{author.name}" CREATE (b)-[r:WRITTEN]->(a)')
    for article in articles:
        for i in article.field:
            session.run(f'MATCH (a:field),(b:article) WHERE "{i}"= a.name AND "{article.name}"=b.name CREATE (b)-[r:belong]->(a)')
            session.run(f'MATCH (a:field),(b:article) WHERE "{i}"= a.name AND "{article.name}"=b.name CREATE (a)-[r:contain]->(b)')
        for a in article.author:
            for b in article.author:
                if a!=b:
                    session.run(f'MATCH (a:author),(b:author) WHERE a.name="{a}" AND b.name="{b}" CREATE (a)-[r:coauthor {{name:"{article.name}"}}]->(b)')


    for author in authors:
        for a in author.paper_list:
            for b in author.paper_list:
                if a!=b:
                    session.run(f'MATCH (a:article),(b:article) WHERE a.name="{a}" AND b.name="{b}" CREATE (a)-[r:sameauthor {{name:"{author.name}"}}]->(b)')

# 关闭驱动程序
driver.close()

In [15]:
print(articles[0])